In [1]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [2]:
load_dotenv()

True

In [ ]:
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()

# **all-MiniLM-L6-v2** model

In [ ]:
# practice
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_function)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)


In [ ]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

# LangChain Expression with Chroma DB CSV (RAG) - **OpenAIEmbeddings** model

In [3]:
embeddings_ = OpenAIEmbeddings()
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()
db = Chroma.from_documents(documents, embeddings_)

In [4]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

Name: HOT WATER BOTTLE I AM SO POORLY
Quantity: 8
UnitPrice: 4.65


In [5]:
retriever = db.as_retriever()

template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically recommend the product with unit price:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


/Users/choesuna/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
print(chain.invoke("I am thirsty so want to buy some drinks."))
print(chain.invoke("I need a hanger to hang."))
print(chain.invoke("Soon my friend's birthday."))

Based on the given context, you can buy the "VINTAGE BILLBOARD DRINK ME MUG" which has a quantity of 6 and a unit price of 1.06.
Based on the provided context, you can consider purchasing the following hangers:
1. CROCHET ROSE DES CLOTHES HANGER - Quantity: 2, Unit Price: $2.95 each
2. HOME SWEET HOME 3 PEG HANGER - Quantity: 2, Unit Price: $3.75 each
3. DOOR HANGER MUM + DADS ROOM - Quantity: 1, Unit Price: $1.45
4. HOME SWEET HOME 3 PEG HANGER - Quantity: 1, Unit Price: $3.75

You have a variety of hangers to choose from based on your preferences and needs.
Based on the context provided, you could consider buying birthday items such as a "BLUE HAPPY BIRTHDAY BUNTING," "ROBOT BIRTHDAY CARD," "ELEPHANT BIRTHDAY CARD," and "TOAST ITS - HAPPY BIRTHDAY" to celebrate your friend's upcoming birthday.


# Shopping with ai assistant

1. The users ask the list of products they want
2. AI provides the product recommendations & ask the user for the specific product and quantity
3. The AI processes the user's choice and calculates the total price based on the unit price and quantity
4. The AI informs the user of the total cost

In [ ]:
failure_cnt = 0
success_cnt = 0

In [25]:
# from langchain import ChatOpenAI, CSVLoader, Chroma, OpenAIEmbeddings, StrOutputParser, ChatPromptTemplate, RunnablePassthrough

# Step 1: Setup
embeddings_ = OpenAIEmbeddings()
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()
db = Chroma.from_documents(documents, embeddings_)
retriever = db.as_retriever()

# Step 2: Define the initial prompt template
initial_template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically show all the product with unit price:
{context}

Question: {question}
"""

# Step 3: Define the follow-up prompt template
follow_up_template = """
Let me know the name of the product and quantity you want.
"""


# Step 4: Find the selected product
def calculate_total_price(product_name, quantity, product_list):
    for product in product_list:
        if product_name.lower() in product['name'].lower():
            unit_price = product['price']
            total_price = unit_price * quantity
            success_cnt +=1
            return total_price
    failure_cnt += 1
    return None

    
# Step 5: Setting up the chain with the templates and model
initial_prompt = ChatPromptTemplate.from_template(initial_template)
follow_up_prompt = ChatPromptTemplate.from_template(follow_up_template)


In [28]:

model = ChatOpenAI()
# Creating the product list
product_list = []

# Function to execute the conversation chain
def run_conversation(user_question):
    # Get initial recommendations
    initial_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | initial_prompt
        | model
        | StrOutputParser()
    )

    initial_response = initial_chain.invoke(user_question)
    print(f"AI: {initial_response}")

    # Ask for product choice
    print("AI: Let me know the name of the product and quantity you want.")
    # Extract product and quantity from user input
    product_name = input("Choose the product- only input the name:")
    quantity = int(input("How many stock do you want? - Just input the number:"))

    # Product list
    docs = db.similarity_search(user_question)
    # Parsing function to extract product details
    def parse_product(doc):
        lines = doc.page_content.split('\n')
        product_info = {}
        for line in lines:
            key, value = line.split(': ')
            key = key.strip().lower()
            value = value.strip()
            if key == 'name':
                product_info['name'] = value
            elif key == 'unitprice':
                product_info['price'] = float(value)
            elif key == 'quantity':
                product_info['quantity'] = int(value)
        return product_info


    for doc in docs:
        product_info = parse_product(doc)
        if product_info not in product_list:
            product_list.append(product_info)
    
    

    # Calculate total price
    total_price = calculate_total_price(product_name.strip(), quantity, product_list)
    
    if total_price is not None:
        print(f"AI: The price of {product_name.strip()} is ${total_price}.")
    else:
        print("AI: Sorry, I couldn't find the product you specified.")

# Step 6: Running the conversation with the user question
user_input = input("Ask what you want :)")
print(f"User: {user_input}")
run_conversation(user_input)

User: soon my friend's Birthday. can you recommend somethings
AI: Sure! For your friend's birthday, I recommend the CARD SUKI BIRTHDAY. It is priced at $0.42 per unit.
AI: Let me know the name of the product and quantity you want.
AI: The price of CARD SUKI BIRTHDAY is $0.42.


# Evaluation

In [29]:
# Print product list to verify
for product in product_list:
    print(product)

{'name': 'SPACEBOY BIRTHDAY CARD', 'quantity': 12, 'price': 0.42}
{'name': 'ELEPHANT, BIRTHDAY CARD,', 'quantity': 1, 'price': 0.85}
{'name': 'CARD SUKI BIRTHDAY', 'quantity': 12, 'price': 0.42}


# Question list
- I am thirsty so want to buy some drinks.
- Is there any wallet?
- Soon my friend's birthday.